In [25]:
import numpy as np
import pandas as pd
import json
from importlib import reload 
import statsmodels.api as sm
import bright_future_base as bf
reload(bf)

<module 'bright_future_base' from '/home/shinjini/Shinjini/MIT/Courses/Spring 2022/6.S079/Project/Code/bright-future/src/bright_future_base.py'>

# Example 1 with BrightFuture

In [26]:
df_works_author = bf.load_df("works", grouped_by="author")
df_awards_author = bf.load_df("awards", grouped_by="author")
bf.merge_datasets(["works", "awards"])

,author,adjustedcount,count,award_count
0,,0.000000,0.0,0.0
1,A Min Tjoa,0.533330,2.0,0.0
2,A. Aldo Faisal,2.309520,8.0,0.0
3,A. B. M. Alim Al Islam,0.529759,3.0,0.0
4,A. B. Siddique,0.999990,3.0,0.0
5,A. Cüneyd Tantug,0.666660,2.0,0.0
6,A. David Marshall,1.666650,5.0,0.0
7,A. E. Eiben,0.500000,1.0,0.0
8,A. Ercüment Çiçek,0.666660,2.0,0.0
9,A. Jefferson Offutt,1.833330,5.0,0.0


# Example 1 without BrightFuture

In [27]:
df = pd.read_csv('../data/awards-data.tsv', sep='\t', header=0)

with open('../data/csrankings/authors-small.json') as f:
    authors_small = json.load(f)
df_authors = pd.DataFrame.from_dict(authors_small)
df_authors['count'] = df_authors['count'].astype(float)
df_authors['adjustedcount'] = df_authors['adjustedcount'].astype(float)
df_authors['year'] = df_authors['year'].astype(float)
df_authors = df_authors.rename(columns={"title": "field", "name": "author"})
df_works = df_authors

df_works_author = df_works.groupby("author").sum().reset_index()
df_awards = df.assign(
    award_count=1).groupby("author-name").sum().drop(
    columns=["year"]).reset_index()

merged_df = df_awards.merge(df_works_author,
                            left_on='author-name',
                            right_on='author',
                            how='right')

merged_df = merged_df.fillna(0).drop(columns=["year"])


mod = sm.OLS(merged_df.award_count,
             sm.add_constant(merged_df[["count"]]))
res = mod.fit()
print(res.summary())


mod = sm.OLS(merged_df.award_count,
             sm.add_constant(merged_df[["adjustedcount",
                                        "count"]]))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            award_count   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     11.21
Date:                Fri, 13 May 2022   Prob (F-statistic):              0.185
Time:                        01:01:55   Log-Likelihood:                 1.7531
No. Observations:                   3   AIC:                            0.4938
Df Residuals:                       1   BIC:                            -1.309
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2540      0.589      3.825      0.1

/home/shinjini/.local/lib/python3.6/site-packages/statsmodels/stats/stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)
/home/shinjini/.local/lib/python3.6/site-packages/statsmodels/stats/stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)


not finished yet but here's the part of model builder that runs model, chooses features...

In [ ]:
def model_builder(y, X, thresh):
    features_choices = ["adjustedcount", "count"]

    features_chosen = []
    features_prospective = []

    res_best = None
    # r2_best_chosen = 0
    res_best_prospective = None
    # r2_best_prospective = 0
    features_best_prospective = None

    # Iteratively build a model. (stop if the model already has all the possible features)
    while len(features_chosen) < len(features_choices):
        # Given the existing model, we test adding each new future, see if there's improvement, and what the best is.

        # First, test adding each new feature to the existing model
        r2_best_prospective = 0
        for feature in features_choices:
            # Only add _new_ features
            if feature not in features_chosen:
                # build features from the old set, add one
                features_prospective = features_chosen.copy()
                features_prospective.append(feature)
                mod = sm.OLS(merged_df.award_count, sm.add_constant(merged_df[features_prospective]))
                res = mod.fit()

                if res.rsquared > res_best_prospective.rsquared:
                    r2_best_prospective = res.rsquared
                    features_best_prospective = features_prospective

        # now we have the best prospective model, and its r2.
        # if this represents a 10% bump in r2 then use it


        if r2_best_prospective >= 1.1 * r2_best_chosen:
            # use it! track its score too
            r2_best_chosen = r2_best_prospective
            features_chosen = features_best_prospective
        else:
            # not a sufficient boost. terminate with the previous best; this round of search produced nothing
            break




# Example 2 with BrightFuture

In [6]:
df_prof = bf.load_df("profs")

# Example 2 without BrightFuture

In [5]:
df_prof = pd.read_csv('../data/profs.tsv', sep='\t', header=0)

uni_rankings = ["Massachusetts Institute of Technology",
                "Carnegie Mellon University",
                "Stanford University",
                "University of California, Berkeley",
                "University of Illinois at Urbana-Champaign",
                "Cornell University",
                "Georgia Institute of Technology",
                "University of Washington",
                "Princeton University",
                "University of Texas at Austin"]

df_prof["is_uni_top_10"] = df_prof["University"].isin(uni_rankings).astype(int)
df_prof["is_bachelors_top_10"] = df_prof["Bachelors"].isin(uni_rankings).astype(int)
df_prof["is_doctorate_top_10"] = df_prof["Doctorate"].isin(uni_rankings).astype(int)

mod = sm.OLS(df_prof.is_uni_top_10,
             sm.add_constant(df_prof[["is_bachelors_top_10",
                                      "is_doctorate_top_10"]]))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:          is_uni_top_10   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     190.5
Date:                Fri, 13 May 2022   Prob (F-statistic):           1.59e-80
Time:                        00:45:55   Log-Likelihood:                -2424.5
No. Observations:                5144   AIC:                             4855.
Df Residuals:                    5141   BIC:                             4875.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.1126    

In [29]:
reload(bf)

<module 'bright_future_base' from '/home/shinjini/Shinjini/MIT/Courses/Spring 2022/6.S079/Project/Code/bright-future/src/bright_future_base.py'>

In [23]:
bf.load_df("works", "area")

,area,adjustedcount,count,year
0,aaai,3752.700521,12398.0,18093230.0
1,acl,1913.456628,5950.0,8172398.0
2,ase,384.816258,1444.0,2525856.0
3,asplos,389.425548,1589.0,2874771.0
4,cav,533.834349,1637.0,2806405.0
5,ccs,815.256148,3150.0,5343885.0
6,chiconf,2003.367587,7612.0,10536005.0
7,crypto,775.755032,2132.0,3393298.0
8,cvpr,4018.326859,14046.0,17120564.0
9,dac,1011.550825,3496.0,5271318.0
